El código se debe correr después de entrenar la red neuronal del archivo principal

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

In [ ]:
%cd /content/MyDrive/My Drive/

In [ ]:
%cd face-mask-detector

In [ ]:
# librerias 
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os
from google.colab.patches import cv2_imshow

In [ ]:
# librerias 
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
#detección de caras usando un modelo pre-entrenado
net=cv2.dnn.readNet('face_detector/deploy.prototxt','face_detector/res10_300x300_ssd_iter_140000.caffemodel')

In [ ]:
face_recognition=net

In [ ]:
#JavaScript - crear la conexion para obtener el video de la camara
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
def detect_and_predict_mask(frame, faceNet, maskNet):
	# toma las dimensiones del marco y luego construye un blob
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
		(104.0, 177.0, 123.0))

	# pasa el blob a través de la red y obtener las detecciones de rostros
	faceNet.setInput(blob)
	detections = faceNet.forward()

	# inicializa la lista de caras, sus ubicaciones correspondientes,
	# y la lista de predicciones 
	faces = []
	locs = []
	preds = []

	# loop over the detections
	for i in range(0, detections.shape[2]):
		# probabilidad de confianza
		confidence = detections[0, 0, i, 2]

		# filtrar las detecciones débiles
		if confidence > 0.5:
			# coordenadas del bounding box
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# dimensiones bounding 
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# preproceso
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)

			# agrega la cara y los cuadros delimitadores respectivamente
			faces.append(face)
			locs.append((startX, startY, endX, endY))

	# hace la prediccion si se detecto al menos una cara
	if len(faces) > 0:
    
		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=32)

  #devuelve una tupla 2 de las ubicaciones de las caras y sus correspondientes ubicaciones
	return (locs, preds)

In [ ]:
def js_reply_to_image(js_reply):
    """
    input: 
          js_reply: JavaScript object, contain image from webcam
    output: 
          image_array: image array RGB size 512 x 512 from webcam
    """
    jpeg_bytes = base64.b64decode(js_reply['img'].split(',')[1])
    image_PIL = Image.open(io.BytesIO(jpeg_bytes))
    image_array = np.array(image_PIL)

    return image_array


# camara web
video_stream()
label_html = 'Video ...'
# inicializar valores 
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    #convierte imagen de entrada de javascript a imagen OpenCV 
    img = js_to_image(js_reply["img"])

    # crear una bounding box y transparencia
    bbox_array = np.zeros([400,500,4], dtype=np.uint8)

    jpeg_bytes = base64.b64decode(js_reply['img'].split(',')[1])
    image_PIL = Image.open(io.BytesIO(jpeg_bytes))
    image_array = np.array(image_PIL)

    frame = img
    v=True
    if v == True:

      # frame = imutils.resize(frame, width=400)
      frame = imutils.resize(frame, width=500)
      # detectar rostros
      # modelo para cubrebocas
      (locs, preds) = detect_and_predict_mask(frame, faceNet, model)
      for (box, pred) in zip(locs, preds):


      # cuadro delimitador y las predicciones
        (startX, startY, endX, endY) = box
        ( bad_mask, mask, withoutMask) = pred
          
        if bad_mask > mask and bad_mask > withoutMask:
              label = "Bad mask" 
        elif mask > bad_mask and mask > withoutMask:
              label = "Mask"
        else: 
              label= "No Mask"
        if label=="Bad mask":
              color=(255, 132, 0) 
        elif label=="Mask":
              color = (0, 255, 0) 
        else:
            color = (255, 0, 0)
        # confianza
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        #mostrar el cuadro y las etiquetas con el resultado
        bbox_array=cv2.putText(bbox_array, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        bbox_array=cv2.rectangle(bbox_array, (startX, startY), (endX, endY), color, 2)
        bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
        bbox_bytes = bbox_to_bytes(bbox_array)
        bbox = bbox_bytes
